In [18]:
# import dependencies
import bs4
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
# from selenium import webdriver
# from lxml import html
import csv
from datetime import datetime
from pytz import timezone

In [2]:
# url of ucsc college 9 & 10 dining hall
url = 'https://nutrition.sa.ucsc.edu/nutframe.asp?sName=UC+Santa+Cruz+Dining&locationNum=40&locationName=Colleges+Nine+%26+Ten+Dining+Hall&naFlag=1'

In [4]:
# scrap the menu data from the webpage
# since the menu data is cannot be scrapped from the html file directly
# we have to make additional requests to get the frame page contents 
with requests.Session() as session:
    response = session.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    for frame in soup.select("frameset frame"):
        frame_url = urljoin(url, frame["src"])
        response = session.get(frame_url)
        frame_soup = BeautifulSoup(response.content, 'html.parser') 
        # print(frame_soup.prettify())

In [5]:
# extract both the meal name and food name from html
meal_name = frame_soup.find_all('div', attrs={'class': 'menusampmeals'})
food_name = frame_soup.find_all('div', attrs={'class': 'menusamprecipes'})

In [6]:
# convert the name of the meal type in html form to a list form
meal_type = [item.string for item in meal_name]
print(meal_type)

['Breakfast', 'Lunch', 'Dinner']


In [7]:
# convert the food name in html form to a list form
food_list = [item.string for item in food_name]
print(food_list)

['Blueberry Pancakes', 'Cage Free Scrambled Eggs', 'Hard-boiled Cage Free Eggs', 'Natural BridgesTofu Scramble', 'Oatmeal Gluten-Free', 'Sausage Patties', 'Steamed Rice', 'Tator Tots', 'N.E.Clam Chowder', 'Baked Salmon with Avocado Chimichurri', 'Black Rice, Beet and Kale Salad', 'Oven Roasted Allergen Free Chicken Thigh', 'Steamed Rice', 'Steamed Seasonal Vegetables', 'Armenian Salad', 'Beef and Lamb Gyro', 'Falafels', 'Tabouleh Salad', 'Tahini Sauce', 'Tzatziki Sauce', "Chef's Special", 'Oven Roasted Allergen Free Chicken Thigh', 'Steamed Rice', 'Steamed Seasonal Vegetables', 'Cajun Bayou Bar', 'Cajun Coleslaw', 'Cajun Rice', 'Chicken Tenders', 'Corn and Okra Maque Choux', 'Creole Style Red Beans and Vegetables', 'Spicy Cajun Gravy']


In [8]:
# preference list, labeled by myself
pref = [1, 1, 1, 0, 0, 
		1, 0, 0, 0, 1, 
		1, 0, 0, 1, 0, 
		0, 0, 0, 0, 1, 
		1, 0, 0, 0, 0,
		1, 0, 0, 0 ]

In [21]:
# timezone setting
us_pacific = timezone('US/Pacific')
cn_time = datetime.now(us_pacific)
time = cn_time.strftime('%Y-%m-%d')

In [22]:
# write the food list to a csv file
csv_path = '/Users/7w0r4ng3s/Desktop/menu_scraping/{}.csv'.format(time)

with open(csv_path, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for key, val in zip(food_list, pref):
        writer.writerow([key, val])    